In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

Exercise 8.

In [3]:
default_reber_grammar = [
    [('B', 1)],
    [('T', 2), ('P', 3)],
    [('S', 2), ('X', 4)],
    [('T', 3), ('V', 5)],
    [('X', 3), ('S', 6)],
    [('P', 4), ('V', 6)],
    [('E', None)]
]

embedded_reber_grammar = [
    [('B', 1)],
    [('T', 2), ('P', 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [('T', 6)],
    [('P', 6)],
    [('E', None)]
]

In [4]:
def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return ''.join(output)        

In [5]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end=' ')

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

In [6]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=' ')

BTBPTTTVPXTVPXTTVPSETE BPBPTVPSEPE BPBPVVEPE BPBPVPXVVEPE BPBTXXTTTTVVEPE BPBPVPSEPE BPBTXXVPSEPE BPBTSSSSSSSXSEPE BTBPVVETE BPBTXXVVEPE BPBTXXVPSEPE BTBTXXVVETE BPBPVVEPE BPBPVVEPE BPBTSXSEPE BPBPVVEPE BPBPTVPSEPE BPBTXXVVEPE BTBPTVPXVVETE BTBPVVETE BTBTSSSSSSSXXVVETE BPBTSSSXXTTTTVPSEPE BTBPTTVVETE BPBTXXTVVEPE BTBTXSETE 

In [7]:
POSSIBLE_CHARS = 'BEPSTVX'

def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

In [8]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=' ')

BTBPTTTPPXTVPXTTVPSETE BPBTXEEPE BPBPTVVVEPE BPBTSSSSXSETE BPTTXSEPE BTBPVPXTTTTTTEVETE BPBTXXSVEPE BSBPTTVPSETE BPBXVVEPE BEBTXSETE BPBPVPSXPE BTBPVVVETE BPBTSXSETE BPBPTTTPTTTTTVPSEPE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE 

In [9]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [10]:
string_to_ids('BTBPTTTPPXTVPXTTVPSETE')

[0, 4, 0, 2, 4, 4, 4, 2, 2, 6, 4, 5, 2, 6, 4, 4, 5, 2, 3, 1, 4, 1]

In [11]:
def generate_dataset(size):
    good_strings = [string_to_ids(generate_string(embedded_reber_grammar)) 
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted_string(embedded_reber_grammar)) 
                   for _ in range(size - (size // 2))]
    all_strings = good_strings + bad_strings
    x = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] 
                 + [[0.] for _ in range(len(bad_strings))])
    return x, y

In [12]:
np.random.seed(42)

x_train, y_train = generate_dataset(10000)
x_valid, y_valid = generate_dataset(2000)

In [13]:
x_train[0]

<tf.Tensor: shape=(22,), dtype=int32, numpy=array([0, 4, 0, 2, 4, 4, 4, 5, 2, 6, 4, 5, 2, 6, 4, 4, 5, 2, 3, 1, 4, 1])>

In [14]:
y_train[0]

array([1.])

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

embedding_size = 5

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation='sigmoid')
])

optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.95, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
313/313 [==============================] - 35s 106ms/step - loss: 0.6910 - acc: 0.5095 - val_loss: 0.6825 - val_acc: 0.5645
Epoch 2/20
313/313 [==============================] - 33s 106ms/step - loss: 0.6678 - acc: 0.5659 - val_loss: 0.6635 - val_acc: 0.6105
Epoch 3/20
313/313 [==============================] - 36s 117ms/step - loss: 0.6504 - acc: 0.5766 - val_loss: 0.6521 - val_acc: 0.6110
Epoch 4/20
313/313 [==============================] - 36s 114ms/step - loss: 0.6347 - acc: 0.5980 - val_loss: 0.6224 - val_acc: 0.6445
Epoch 5/20
313/313 [==============================] - 34s 109ms/step - loss: 0.6054 - acc: 0.6361 - val_loss: 0.5779 - val_acc: 0.6980
Epoch 6/20
313/313 [==============================] - 34s 108ms/step - loss: 0.5414 - acc: 0.7093 - val_loss: 0.4695 - val_acc: 0.7795
Epoch 7/20
313/313 [==============================] - 35s 111ms/step - loss: 0.4141 - acc: 0.8207 - val_loss: 0.4368 - val_acc: 0.8050
Epoch 8/20
313/313 [==============================] - 3

In [16]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]

x_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], 
                            ragged_rank=1)
y_proba = model.predict(x_test)
print('Estimated probability that these are Reber strings:')
for index, string in enumerate(test_strings):
    print(f'\t{string}: {y_proba[index][0] * 100:.2f}%')

Estimated probability that these are Reber strings:
	BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: 0.04%
	BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: 99.98%
